## **Step 1: Install Required Libraries**

In [9]:
!pip install spacy tqdm

## **Step 2: Download the spaCy Model**

In [10]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.8 MB 2.5 MB/s eta 0:00:05
     ----- ---------------------------------- 1.8/12.8 MB 2.5 MB/s eta 0:00:05
     ------- -------------------------------- 2.4/12.8 MB 2.4 MB/s eta 0:00:05
     --------- ------------------------------ 2.9/12.8 MB 2.5 MB/s eta 0:00:05
     ---------- ----------------------------- 3.4/12.8 MB 2.5 MB/s eta 0:00:04
     ------------ --------------------------- 3.9/12.8 MB 2.4 MB/s eta 0:00:04
     ------------- -------------------------- 4.5/12.8 MB 2.4 MB/s eta 0:00:04
     --------------- ------------------------ 5.0/12.8 MB 2.5 MB/s eta 0:00:04
     ----------------- ---------------------- 5.5/12.8 MB 2.4 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 2.4 MB/s

## **1️. Define Entities for NER**

Since you want to recognize which product a customer is asking about, you need to identify relevant entities.
For example:

Product Names (PRODUCT) → "Dental EMR", "Practice Management Software"

Service Types (SERVICE) → "Revenue Cycle Management", "Medical Billing"

Company Name (COMPANY) → "SRS Web Solutions"

## **2️. Choose an NER Approach**

You have two main options:

🔹 Option 1: Pre-trained NER Model (Fast but Less Accurate)
1. Use a Hugging Face model like Blaze999/Medical-NER or dbmdz/bert-large-cased-finetuned-conll03-english.

2. Extract entities from customer queries.

3. Pros: Quick setup

4. Cons: May not recognize your custom products well.

🔹 Option 2: Custom NER Model (Best for Accuracy)

1. Train a spaCy NER or Fine-tune BERT with labeled sales data.

2. Examples:

    . SRS Web Solutions offers Dental EMR. → { "text": "Dental EMR", "label": "PRODUCT" }

    .  I need help with medical billing. → { "text": "medical billing", "label": "SERVICE" }

3. Pros: High accuracy for your products

4. cons: Needs training data.

## **3️. Implement a Basic NER Pipeline**

Here’s a simple NER setup using spaCy:

In [8]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
import random
import os
from tqdm import tqdm

# Try to import tqdm for progress bars, fall back to a simple function if not available
try:
    from tqdm import tqdm
except ImportError:
    # Simple function to use if tqdm is not available
    def tqdm(iterable, **kwargs):
        print(f"Starting {kwargs.get('desc', 'process')}...")
        return iterable

# Expanded training data with variations
TRAIN_DATA = [
    # Original examples
    ("I want to buy Dental EMR.", {"entities": [(16, 27, "PRODUCT")]}),
    ("Does your company sell Dental EMR software?", {"entities": [(24, 34, "PRODUCT")]}),
    ("Can I get details on Practice Management Software?", {"entities": [(21, 49, "PRODUCT")]}),
    ("Tell me about your Revenue Cycle Management service.", {"entities": [(19, 48, "SERVICE")]}),
    ("Does your service include Medical Billing?", {"entities": [(26, 40, "SERVICE")]}),
    ("What are the benefits of Telemedicine Solutions?", {"entities": [(24, 47, "PRODUCT")]}),
    ("I'm interested in a cloud-based Dental EMR system.", {"entities": [(28, 38, "PRODUCT")]}),
    ("How does your Medical Billing software work?", {"entities": [(12, 27, "SERVICE")]}),
    ("Can you provide a demo of Revenue Cycle Management?", {"entities": [(25, 54, "SERVICE")]}),
    ("Do you offer cloud-based Telemedicine Solutions?", {"entities": [(23, 46, "PRODUCT")]}),
    
    # Additional examples to improve detection
    ("Tell me about Dental EMR.", {"entities": [(14, 25, "PRODUCT")]}),
    ("I need information on Dental EMR products.", {"entities": [(21, 32, "PRODUCT")]}),
    ("Telemedicine Solutions are becoming popular.", {"entities": [(0, 23, "PRODUCT")]}),
    ("Revenue Cycle Management improves cashflow.", {"entities": [(0, 29, "SERVICE")]}),
    ("Medical Billing services can reduce costs.", {"entities": [(0, 15, "SERVICE")]}),
    ("We specialize in Practice Management Software.", {"entities": [(17, 45, "PRODUCT")]}),
    ("Is Dental EMR included in your package?", {"entities": [(3, 14, "PRODUCT")]}),
    ("How much does the Telemedicine Solutions cost?", {"entities": [(18, 41, "PRODUCT")]}),
    ("Are your Revenue Cycle Management tools cloud-based?", {"entities": [(9, 38, "SERVICE")]}),
    ("Medical Billing is essential for healthcare providers.", {"entities": [(0, 15, "SERVICE")]})
]

# Function to create and train NER model
def train_healthcare_ner(use_pretrained=False):
    # Create NLP object - try to use pretrained model if requested
    if use_pretrained:
        try:
            print("Attempting to load pretrained model...")
            nlp = spacy.load("en_core_web_sm")
            print("Pretrained model loaded successfully!")
        except Exception as e:
            print(f"Could not load pretrained model: {e}")
            print("Falling back to blank model...")
            nlp = spacy.blank("en")
    else:
        print("Using blank model as requested...")
        nlp = spacy.blank("en")
    
    # Add NER pipeline if not present
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
        print("Added NER pipeline component.")
    else:
        ner = nlp.get_pipe("ner")
        print("Using existing NER pipeline component.")
    
    # Add entity labels
    print("Adding entity labels...")
    labels = set()
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            labels.add(ent[2])
    
    for label in labels:
        ner.add_label(label)
    print(f"Added {len(labels)} labels: {', '.join(labels)}")
    
    # Split data into training and evaluation sets (80/20 split)
    print("Splitting data into training and evaluation sets...")
    random.shuffle(TRAIN_DATA)
    split = int(len(TRAIN_DATA) * 0.8)
    train_data = TRAIN_DATA[:split]
    eval_data = TRAIN_DATA[split:]
    print(f"Training data: {len(train_data)} examples")
    print(f"Evaluation data: {len(eval_data)} examples")
    
    # Configure training parameters
    n_iter = 100
    batch_size = 4
    
    # Disable other pipelines during training for better performance
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    print(f"Disabling other pipelines during training: {', '.join(other_pipes) if other_pipes else 'none'}")
    
    # Train with better parameters
    with nlp.disable_pipes(*other_pipes):
        # Only train NER
        optimizer = nlp.begin_training()
        print(f"Starting training for {n_iter} iterations...")
        
        # Training loop
        for i in range(n_iter):
            # Shuffle training data
            random.shuffle(train_data)
            batches = list(spacy.util.minibatch(train_data, size=batch_size))
            losses = {}
            
            # Process batches with progress bar
            for batch in tqdm(batches, desc=f"Iteration {i+1}/{n_iter}"):
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    examples.append(example)
                
                # Update with appropriate dropout (higher to prevent overfitting)
                nlp.update(
                    examples,
                    drop=0.3,
                    losses=losses,
                    sgd=optimizer
                )
            
            # Print loss for this iteration
            print(f"Iteration {i+1}, Loss: {losses}")
            
            # Evaluate on the evaluation set every 10 iterations
            if i % 10 == 9 or i == n_iter - 1:
                # Calculate evaluation metrics
                correct = 0
                total_gold = 0
                total_pred = 0
                
                for text, annot in eval_data:
                    doc_gold_text = nlp.make_doc(text)
                    gold = Example.from_dict(doc_gold_text, annot)
                    pred_value = nlp(text)
                    
                    gold_ents = set([(ent.text, ent.label_) for ent in gold.reference.ents])
                    pred_ents = set([(ent.text, ent.label_) for ent in pred_value.ents])
                    
                    correct += len(gold_ents & pred_ents)
                    total_gold += len(gold_ents)
                    total_pred += len(pred_ents)
                
                precision = correct / total_pred if total_pred > 0 else 0
                recall = correct / total_gold if total_gold > 0 else 0
                f_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
                
                print(f"Evaluation - Precision: {precision:.2f}, Recall: {recall:.2f}, F-score: {f_score:.2f}")
    
    # Save the trained model
    output_dir = "healthcare_ner_model"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    nlp.to_disk(output_dir)
    print(f"Model saved to {output_dir}")
    
    return nlp, output_dir

# Test function with detailed output
def test_ner_model(nlp, test_sentences):
    print("\nTesting NER model...\n" + "-" * 50)
    
    for sentence in test_sentences:
        doc = nlp(sentence)
        
        # Print sentence with highlighted entities
        tokens = []
        for token in doc:
            if token.ent_type_:
                tokens.append(f"[{token.text}:{token.ent_type_}]")
            else:
                tokens.append(token.text)
        
        highlighted = " ".join(tokens)
        
        print(f"\nInput: {sentence}")
        print(f"Parsed: {highlighted}")
        print(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")
        print("-" * 50)

# Main execution
if __name__ == "__main__":
    # First try with blank model - this will always work
    print("=" * 70)
    print("TRAINING WITH BLANK MODEL")
    print("=" * 70)
    nlp, model_dir = train_healthcare_ner(use_pretrained=False)
    
    # Test sentences
    test_sentences = [
        "Tell me about Dental EMR.",
        "Do you provide Telemedicine Solutions?",
        "I need help with Revenue Cycle Management.",
        "What are the features of Practice Management Software?",
        "Does your company offer Medical Billing?",
        # New sentences
        "Our clinic uses a different Dental EMR platform.",
        "Can we integrate your Medical Billing with our system?",
        "How secure is your Telemedicine Solutions platform?",
    ]
    
    # Test the model
    test_ner_model(nlp, test_sentences)
    
    print("\nTo install the spaCy model for better results next time, run:")
    print("python -m spacy download en_core_web_sm")

TRAINING WITH BLANK MODEL
Using blank model as requested...
Added NER pipeline component.
Adding entity labels...
Added 2 labels: SERVICE, PRODUCT
Splitting data into training and evaluation sets...
Training data: 16 examples
Evaluation data: 4 examples
Disabling other pipelines during training: none
Starting training for 100 iterations...


Iteration 1/100: 100%|██████████| 4/4 [00:00<00:00, 41.05it/s]


Iteration 1, Loss: {'ner': 85.57232841104269}


Iteration 2/100: 100%|██████████| 4/4 [00:00<00:00, 42.25it/s]


Iteration 2, Loss: {'ner': 67.53044295310974}


...

Iteration 98, Loss: {'ner': 0.00018222966133523062}


Iteration 99/100: 100%|██████████| 4/4 [00:00<00:00, 41.46it/s]


Iteration 99, Loss: {'ner': 8.852585313493656e-08}


Iteration 100/100: 100%|██████████| 4/4 [00:00<00:00, 41.81it/s]

Iteration 100, Loss: {'ner': 4.666899653841235e-07}
Evaluation - Precision: 0.00, Recall: 0.00, F-score: 0.00
Model saved to healthcare_ner_model

Testing NER model...
--------------------------------------------------

Input: Tell me about Dental EMR.
Parsed: Tell me about [Dental:PRODUCT] [EMR:PRODUCT] [.:PRODUCT]
Entities: [('Dental EMR.', 'PRODUCT')]
--------------------------------------------------

Input: Do you provide Telemedicine Solutions?
Parsed: Do you provide Telemedicine Solutions ?
Entities: []
--------------------------------------------------

Input: I need help with Revenue Cycle Management.
Parsed: I need help with Revenue [Cycle:PRODUCT] [Management:PRODUCT] [.:PRODUCT]
Entities: [('Cycle Management.', 'PRODUCT')]
--------------------------------------------------

Input: What are the features of Practice Management Software?
Parsed: What are the features of [Practice:PRODUCT] [Management:PRODUCT] [Software:PRODUCT] ?
Entities: [('Practice Management Software', 'PR

### **The training and all the tests are for sample, not completed that is why all the entities not show up in the output file**

## **4️. Integrate NER into the Sales Chatbot**
Once you extract the product/service from a customer query, you can:

Recommend relevant features

Route to the correct sales team

Answer FAQs

Example flow:

Customer: "Tell me about Dental EMR."

NER detects "Dental EMR" → PRODUCT

Chatbot fetches product details and responds.

## **5. SUGGESTIONS**

1. Intent Recognition & Classification
   Instead of just identifying keywords, classify the user's intent (e.g., Inquiry, Purchase Request, Pricing, Comparison, Support). You can use:

   Transformer models (BERT, DistilBERT) for intent classification.
   Rule-based + ML hybrid approach (e.g., regex for FAQs + ML for complex queries).

2. Contextual Memory & Conversation Flow
   Make the chatbot remember previous interactions within a session to provide personalized responses.

   Short-term memory (e.g., remembering the user is asking about "Dental EMR" throughout the chat).
   Long-term memory (e.g., if they return later, recall past interactions using a database).

3. Recommendation System
   If a customer inquires about a product, suggest related products dynamically.

   Use collaborative filtering (like Amazon’s recommendations).
   Suggest based on customer history or preferences.

4. Dynamic FAQ Search Using Vector Search (Semantic Search)
   Instead of relying on predefined keyword matches, let the chatbot understand semantic meaning using:

   FAISS / Pinecone for vector-based retrieval.
   BERT-based embeddings to match similar queries intelligently.
   Example: A customer asks, “What are the features of your EMR?” → Instead of static responses, fetch the most relevant FAQ dynamically.

5. Sentiment Analysis & Tone Adjustment
   Detect if the customer is frustrated, happy, or confused and adjust the chatbot’s response tone accordingly.

   Upset users? Offer faster escalation to human support.
   Happy users? Recommend related products or upsell features.

6. Multimodal Chatbot (Voice + Text + Images)
   Instead of just text-based responses:

   Use speech-to-text (Whisper, Vosk, Google Speech API) for voice input.
   Show images/videos of products when customers ask.
   If they upload a document or screenshot, analyze it with OCR (Tesseract, EasyOCR).